<a href="https://colab.research.google.com/github/IMThanhNguyen/NewTest/blob/main/%5BCode_Hint%5D_Project_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://drive.google.com/file/d/1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx/view?usp=drive_link
!gdown 1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx

Downloading...
From: https://drive.google.com/uc?id=1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx
To: /content/YOLOv10_Tutorials.pdf
100% 16.6M/16.6M [00:00<00:00, 39.7MB/s]


## **2. Install and import libraries**

In [16]:
!pip install -q transformers==4.52.4
!pip install -q bitsandbytes==0.46.0
!pip install -q accelerate==1.7.0
!pip install -q langchain==0.3.25
!pip install -q langchainhub==0.1.21
!pip install -q langchain-chroma==0.2.4
!pip install -q langchain_experimental==0.3.4
!pip install -q langchain-community==0.3.24
!pip install -q langchain_huggingface==0.2.0
!pip install -q python-dotenv==1.1.0
!pip install -q pypdf

In [17]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_experimental.text_splitter import SemanticChunker

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

## **3. Read pdf file**

In [24]:
from langchain_community.document_loaders import PyPDFLoader
from google.colab import files
import os

# Upload file
uploaded = files.upload()
FILE_PATH = list(uploaded.keys())[0]  # Get the uploaded filename

# Alternative if you've already uploaded:
# FILE_PATH = "/content/YOLOv10_Tutorials.pdf"

# Verify file exists
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"File not found at {FILE_PATH}")

# Load the document
loader = PyPDFLoader(FILE_PATH)
documents = loader.load()

print(f"Number of documents: {len(documents)}")
print(documents[0])

Saving YOLOv10_Tutorials.pdf to YOLOv10_Tutorials.pdf
Number of documents: 20
page_content='AI VIET NAM – AI COURSE 2024
Tutorial: Phát hiện đối tượng trong ảnh với
YOLOv10
Dinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và
Quang-Vinh Dinh
Ngày 20 tháng 6 năm 2024
I. Giới thiệu
Object Detection (Tạm dịch: Phát hiện đối tượng)là một bài toán cổ điển thuộc lĩnh vực
Computer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong
một tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm
giải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look
Once) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực
thi mà loại mô hình này mang lại.
Hình 1: Logo của mô hình YOLO. Ảnh: link.
Thời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)
đã đề xuất mô hìnhYOLOv10 trong bài báoYOLOv10: Real-Time End-to-End Object
Detection [10]. Với những c

## **4. Initialize text vectorization (embedding)**

In [19]:
embeddings = HuggingFaceEmbeddings(
    model_name="bkai-foundation-models/vietnamese-bi-encoder"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

## **5. Initialize text splitter and split document**

In [25]:
semantic_splitter = SemanticChunker(
    embeddings=embeddings,
    buffer_size=1,                          # Nhóm 3 câu
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
    # number_of_chunks= 30,                   # Số chunks mong muốn. Tìm number_of_chunks - 1 điểm có similarity thấp nhất để cắt. Nếu không đủ điểm cắt phù hợp → số chunks thực tế ≠ số chunks mong muốn
    min_chunk_size=500,                     # Tối thiểu 1000 ký tự
    # sentence_split_regex=r'(?<=[.?!…])\s+', # Bao gồm dấu … cho tiếng Việt
    add_start_index=True
)

docs = semantic_splitter.split_documents(documents)
print("Number of semantic chunks: ", len(docs))

Number of semantic chunks:  32


In [26]:
print(docs[5])
print()
print(docs[6])
print()
print(docs[7])

page_content='Ảnh: Zixuan Zhang. 4' metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-06-20T11:06:10+00:00', 'author': '', 'keywords': '', 'moddate': '2024-06-20T11:06:10+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': 'Overleaf Example', 'trapped': '/False', 'source': 'YOLOv10_Tutorials.pdf', 'total_pages': 20, 'page': 3, 'page_label': '4', 'start_index': 739}

page_content='AI VIETNAM (AIO2024) aivietnam.edu.vn
- Điểm mới: Sử dụng anchor boxes, mạng Darknet-19, và tăng training data để tăng độ
chính xác. - Ưu điểm: Tăng độ chính xác và khả năng nhận diện nhiều object trong 1 cell. - Nhược điểm: Phức tạp hơn, cần nhiều tài nguyên tính toán, và khó detect các object
nhỏ. II.IV. YOLOv3
YOLOv3 [3] ra mắt năm 2018, tiếp tục cải tiến từ YOLOv2. Hình 6: Kiến trúc mô hình YOLOv3. Ảnh: [3]. - Điểm mới: Sử dụng mạng Darknet-53 và detect object ở ba cấp độ k

## **6. Create vector database and retriever**

In [29]:
vector_db = Chroma.from_documents(documents=docs,
                                  embedding=embeddings)

retriever = vector_db.as_retriever()

QUERY = "YOLOv10 dùng để làm gì"
result = retriever.invoke(QUERY)

print("Number of relevant documents: ", len(result))

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Number of relevant documents:  4


In [30]:
print(result[0])
print()
print(result[1])
print()
print(result[2])
print()
print(result[3])

page_content='AI VIETNAM (AIO2024) aivietnam.edu.vn
Hình 2: Hiệu suất của YOLOv10 khi so sánh với các mô hình khác. Trên tập dữ liệu COCO,
YOLOv10 đạt được kết quả tốt nhất về khía cạnh Độ trễ (Latency) và Số lượng tham số mô
hình (Number of parameters) trong khi vẫn giữ được độ chính xác (COCO AP) cao. Ảnh: [10]. Trong bài viết này, chúng ta sẽ cùng nhau tìm hiểu về YOLOv10 và cách sử dụng mô hình này. Thông qua đó, nhóm cũng sẽ trình bày sơ lược về bài toán Object Detection cũng như tóm tắt
ngắn gọn các phiên bản YOLO trước đó để bạn đọc có một cái nhìn tổng quan hơn về nội dung
này. Theo đó, bài viết được bố cục như sau:
- Phần I:Giới thiệu về nội dung bài viết. - Phần II:Tóm tắt về bài toán Object Detection và các phiên bản YOLO đời trước. - Phần III:Trình bày nội dung YOLOv10. - Phần IV:Hướng dẫn cách cài đặt, huấn luyện và sử dụng YOLOv10. - Phần V:Trích dẫn tài liệu. 2' metadata={'page': 1, 'start_index': 0, 'trapped': '/False', 'author': '', 'title': 'Overleaf Example', 'total_

## **7. Load LLMs (Vicuna)**

In [32]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline,
)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Device set to use cuda:0


## **8. Prompting with RAG**

In [33]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
output

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: YOLOv10 là gì? \nContext: YOLOv9\nYOLOv9 [9] được giới thiệu vào năm 2024 bởi Chien-Yao Wang, I-Hau Yeh, và Hong-Yuan Mark\nLiao. Mô hình này cải thiện độ chính xác và tốc độ so với YOLOv8 và giới thiệu nhiều kỹ thuật\nmới như Programmable Gradient Information (PGI) và Generalized Efficient Layer Aggregation\nNetwork (GELAN). 7\n\nYOLOv9\nYOLOv9 [9] được giới thiệu vào năm 2024 bởi Chien-Yao Wang, I-Hau Yeh, và Hong-Yuan Mark\nLiao. Mô hình này cải thiện độ chính xác và tốc độ so với YOLOv8 và giới thiệu nhiều kỹ thuật\nmới như Programmable Gradient Information (PGI) và Generalized Efficient Layer Aggregation\nNetwork (GELAN). 7\n\nYOLOv9\nYOLOv9 [9] được giới thiệu vào năm 2024 bởi Chien-Yao Wang, I-Hau Yeh, và Hong-Yuan Mark\nLiao.

In [34]:
answer = output.split('Answer:')[1].strip()
answer

'YOLOv10 là một phiên bản cải tiến của YOLOv9, được giới thiệu vào năm 2024. Nó cải thiện độ chính xác và tốc độ so với YOLOv8 và giới thiệu nhiều kỹ thuật mới như Programmable Gradient Information (PGI) và Generalized Efficient Layer Aggregation Network (GELAN).'